# 감성 사전을 이용한 영화 리뷰 감성 분석

**NLTK 영화 리뷰 데이터 준비**

In [ ]:
import nltk
nltk.download('movie_reviews')

from nltk.corpus import movie_reviews

In [2]:
print('review count:', len(movie_reviews.fileids()))    # 영화 리뷰 문서의 id를 반환
print('samples of file ids:', movie_reviews.fileids()[:10])     # id를 10개까지만 출력
print('categories of reviews:', movie_reviews.categories())     # label, 즉 긍정인지 부정인지에 대한 분류
print('num of "neg" reviews:', len(movie_reviews.fileids(categories='neg')))    # label이 부정인 문서들의 id를 반환
print('num of "pos" reviews:', len(movie_reviews.fileids(categories='pos')))    # label이 긍정인 문서들의 id를 반환

review count: 2000
samples of file ids: ['neg/cv000_29416.txt', 'neg/cv001_19502.txt', 'neg/cv002_17424.txt', 'neg/cv003_12683.txt', 'neg/cv004_12641.txt', 'neg/cv005_29357.txt', 'neg/cv006_17022.txt', 'neg/cv007_4992.txt', 'neg/cv008_29326.txt', 'neg/cv009_29417.txt']
categories of reviews: ['neg', 'pos']
num of "neg" reviews: 1000
num of "pos" reviews: 1000


In [3]:
fileid = movie_reviews.fileids()[0]     # 첫번째 문서의 id를 반환

print('id of the first review:', fileid)
print('part of the first review:', movie_reviews.raw(fileid)[:500])     # 첫번째 문서의 내용을 500자까지만 출력
print('sentiment of the first review:', movie_reviews.categories(fileid))   # 첫번째 문서의 감성

id of the first review: neg/cv000_29416.txt
part of the first review: plot : two teen couples go to a church party , drink and then drive . 
they get into an accident . 
one of the guys dies , but his girlfriend continues to see him in her life , and has nightmares . 
what's the deal ? 
watch the movie and " sorta " find out . . . 
critique : a mind-fuck movie for the teen generation that touches on a very cool idea , but presents it in a very bad package . 
which is what makes this review an even harder one to write , since i generally applaud films which attempt
sentiment of the first review: ['neg']


In [4]:
fileids = movie_reviews.fileids()       # movie review data에서 file id를 가져옴
reviews = [movie_reviews.raw(fileid) for fileid in fileids]         # file id를 이용해 raw text file을 가져옴
categories = [movie_reviews.categories(fileid)[0] for fileid in fileids] 

**TextBlob을 이용한 감성 분석**

In [ ]:
!pip install -U textblob
!python -m textblob.download_corpora

In [ ]:
from textblob import TextBlob

result = TextBlob(reviews[0])
print(result.sentiment)

Sentiment(polarity=0.06479782948532947, subjectivity=0.5188408350908352)


In [ ]:
# textblob 극성의 범위 [-1.0(부정), 1.0(긍정)], 주관성의 범위 [0.0(객관적), 1.0(주관적)]

def sentiment_TextBlob(docs):
    results = []

    for doc in docs:
        testimonial = TextBlob(doc)
        if testimonial.sentiment.polarity > 0:
            results.append('pos')
        else:
            results.append('neg')
    return results

In [ ]:
from sklearn.metrics import accuracy_score

print('TextBlob을 이용한 리뷰 감성분석의 정확도:', accuracy_score(categories, sentiment_TextBlob(reviews)))

TextBlob을 이용한 리뷰 감성분석의 정확도: 0.6


**AFINN을 이용한 감성 분석**

In [ ]:
!pip install afinn

In [ ]:
from afinn import Afinn

result = Afinn(emoticons=True)
print(result.score(reviews[0]))

-16.0


In [ ]:
def sentiment_Afinn(docs):
    afn = Afinn(emoticons=True)
    results = []

    for doc in docs:
        if afn.score(doc) > 0:
            results.append('pos')
        else:
            results.append('neg')
    return results

In [ ]:
print('Afinn을 이용한 리뷰 감성분석의 정확도:', accuracy_score(categories, sentiment_Afinn(reviews)))

Afinn을 이용한 리뷰 감성분석의 정확도: 0.664


**VADER를 이용한 감성 분석**

In [ ]:
import nltk
nltk.download('vader_lexicon')

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

result = SentimentIntensityAnalyzer()
print(result.polarity_scores(reviews[0]))

{'neg': 0.093, 'neu': 0.762, 'pos': 0.145, 'compound': 0.9924}


In [ ]:
# compound의 범위 [-1.0(부정), 1.0(긍정)], compound <= -0.05: 부정 / -0.05 < compound < 0.05: 중립 / compound >= 0.05: 글정 (일반적인 임계값으로 문서마다 다를 수 있음)
# neg, neu, pos의 태그는 분석할 텍스트 내의 각 태그에 속하는 텍스트의 비율

def sentiment_vader(docs):
    analyser = SentimentIntensityAnalyzer()
    results = []

    for doc in docs:
        score = analyser.polarity_scores(doc)
        if score['compound'] > 0:
            results.append('pos')
        else:
            results.append('neg')

    return results

In [ ]:
print('Vader을 이용한 리뷰 감성분석의 정확도:', accuracy_score(categories, sentiment_vader(reviews)))

Vader을 이용한 리뷰 감성분석의 정확도: 0.635


# 머신러닝 기반의 감성 분석

NLTK 영화 리뷰에 대한 머신러닝 기반 감성 분석<br><br>
**데이터셋 분리**

In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(reviews, categories, test_size=0.2, random_state=7) # 훈련 : 테스트 = 8 : 2

print('Train set count: ', len(X_train))
print('Test set count: ', len(X_test))

Train set count:  1600
Test set count:  400


**Tf-IDF 벡터 만들기**

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer().fit(X_train) 

X_train_tfidf = tfidf.transform(X_train)    # train set을 변환
X_test_tfidf = tfidf.transform(X_test)  # test set을 변환

print('Train set dimension:', X_train_tfidf.shape)     # 실제로 몇개의 특성이 사용되었는지 확인
print('Test set dimension:', X_test_tfidf.shape)

Train set dimension: (1600, 36189)
Test set dimension: (400, 36189)


**나이브 베이즈 분류기**

In [7]:
from sklearn.naive_bayes import MultinomialNB    # sklearn이 제공하는 MultinomialNB 를 사용

NB_clf = MultinomialNB(alpha=0.01)  # 분류기 선언
NB_clf.fit(X_train_tfidf, y_train)  #train set을 이용하여 분류기(classifier)를 학습

print('Train set score: {:.3f}'.format(NB_clf.score(X_train_tfidf, y_train)))  #train set에 대한 예측정확도를 확인
print('Test set score: {:.3f}'.format(NB_clf.score(X_test_tfidf, y_test)))     #test set에 대한 예측정확도를 확인

Train set score: 0.998
Test set score: 0.797


다음 영화 리뷰에 대한 머신러닝 기반 감성 분석<br><br>
JDK & JPype1 & KoNLPy 설치명령어

In [ ]:
%%bash
apt-get update
apt-get install g++ openjdk-8-jdk python-dev python3-dev
pip3 install JPype1
pip3 install konlpy

환경변수 설정

In [ ]:
%env JAVA_HOME "/usr/lib/jvm/java-8-openjdk-amd64"

**데이터셋 확인 및 분리**

In [12]:
import pandas as pd
df = pd.read_csv('/content/drive/Othercomputers/내 컴퓨터/TextMining/data/daum_movie_review.csv')
df.head(5)

,review,rating,date,title
0,돈 들인건 티가 나지만 보는 내내 하품만,1,2018.10.29,인피니티 워
1,몰입할수밖에 없다. 어렵게 생각할 필요없다. 내가 전투에 참여한듯 손에 땀이남.,10,2018.10.26,인피니티 워
2,이전 작품에 비해 더 화려하고 스케일도 커졌지만.... 전국 맛집의 음식들을 한데 ...,8,2018.10.24,인피니티 워
3,이 정도면 볼만하다고 할 수 있음!,8,2018.10.22,인피니티 워
4,재미있다,10,2018.10.20,인피니티 워


In [13]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df.review, df.rating, random_state=7)   # 비율을 지정하지 않으면 75:25로 분할됨

print('Train set size:', len(X_train))
print('Test set size:', len(X_test))

Train set size: 11043
Test set size: 3682


**토크나이저**

In [14]:
from konlpy.tag import Okt  #konlpy에서 Twitter 형태소 분석기를 import

okt = Okt()

def twit_tokenizer(text):   # 전체를 다 사용하는 대신, 명사, 동사, 형용사를 사용
    target_tags = ['Noun', 'Verb', 'Adjective']
    result = []
    for word, tag in okt.pos(text, norm=True, stem=True):
        if tag in target_tags:
            result.append(word)
    return result

**Tf-IDF 벡터 만들기**

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(tokenizer=twit_tokenizer, max_features=2000, min_df=5, max_df=0.5)

X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

print('Train set dimension:', X_train_tfidf.shape)     # 실제로 몇개의 특성이 사용되었는지 확인
print('Test set dimension:', X_test_tfidf.shape)

Train set dimension: (11043, 2000)
Test set dimension: (3682, 2000)


**로지스틱 회귀분석**

In [18]:
from sklearn.metrics import accuracy_score      # 정확도 (올바르게 예측된 데이터의 수를 전체 데이터의 수로 나눈 값)
from sklearn.metrics import precision_score     # 정밀도 (모델이 True로 예측한 데이터 중 실제로 True인 데이터의 수)
from sklearn.metrics import recall_score        # 재현율 (실제로 True인 데이터를 모델이 True라고 인식한 데이터의 수)
from sklearn.metrics import f1_score            # f1 척도 (precision 과 recall의 조화평균)

from sklearn.linear_model import LogisticRegression

In [19]:
# target 값 평점 -> 극성(true, false), 긍정(true)만 선별
y_train_sentiment = (y_train > 5)
y_test_sentiment = (y_test > 5)

LR_clf = LogisticRegression()   # 로지스틱 회귀분석
LR_clf.fit(X_train_tfidf, y_train_senti)    # train data를 이용하여 분류기를 학습

y_train_predict = LR_clf.predict(X_train_tfidf)
y_test_predict = LR_clf.predict(X_test_tfidf)

print('Accuracy for train set: {:.3f}'.format(accuracy_score(y_train_senti, y_train_predict)))
print('Precision for train set: {:.3f}'.format(precision_score(y_train_senti, y_train_predict)))
print('Recall for train set: {:.3f}'.format(recall_score(y_train_senti, y_train_predict)))
print('F1 for train set: {:.3f}\n'.format(f1_score(y_train_senti, y_train_predict)))

print('Accuracy for test set: {:.3f}'.format(accuracy_score(y_test_senti, y_test_predict)))
print('Precision for test set: {:.3f}'.format(precision_score(y_test_senti, y_test_predict)))
print('Recall for test set: {:.3f}'.format(recall_score(y_test_senti, y_test_predict)))
print('F1 for test set: {:.3f}'.format(f1_score(y_test_senti, y_test_predict)))

Accuracy for train set: 0.878
Precision for train set: 0.878
Recall for train set: 0.973
F1 for train set: 0.923

Accuracy for test set: 0.855
Precision for test set: 0.866
Recall for test set: 0.958
F1 for test set: 0.910
